In [5]:
import pandas as pd

df_hist = pd.read_csv('layoffs_historical.csv')

df_hist = df_hist[['company', 'industry', 'total_laid_off', 'date', 'country']]

# Convert Date to standard datetime objects
df_hist['date'] = pd.to_datetime(df_hist['date'])

print("Historical Data Loaded. Shape:", df_hist.shape)
print(df_hist.head())

Historical Data Loaded. Shape: (4253, 5)
          company   industry  total_laid_off       date        country
0         Sapiens    Finance           700.0 2025-12-28         Israel
1       Yellow.ai    Support           100.0 2025-12-23  United States
2  The Trade Desk  Marketing             NaN 2025-12-17  United States
3          Amazon     Retail            84.0 2025-12-15  United States
4     PowerSchool  Education             NaN 2025-12-12  United States


In [7]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager



In [8]:
def get_2026_data_resilient():
    # 1. Setup Driver
    options = Options()
    # options.add_argument("--headless") # Keep off to watch for Captchas
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    url = "https://peerlist.io/layoffs-tracker"
    
    try:
        print(f"Opening {url}...")
        driver.get(url)

        # 2. WAIT for the table to appear (up to 20 seconds)
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))
        
        # 3. Parse with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table')
        
        if table is None:
            print("❌ Error: Table still not found after waiting.")
            return None

        # 4. Extract Data
        rows = table.find_all('tr')
        data = []
        for row in rows[1:]: # Skip header
            cols = row.find_all('td')
            if len(cols) >= 4:
                data.append({
                    'Company': cols[0].text.strip(),
                    'Laid_Off_Count': cols[1].text.strip().split(' ')[0],
                    'Date': cols[2].text.strip(),
                    'Industry': cols[3].text.strip(),
                    'Country': cols[4].text.strip() if len(cols) > 4 else "N/A"
                })
        
        df = pd.DataFrame(data)
        df.to_csv("peerlist_2026.csv", index=False)
        print(f"✅ Success! Scraped {len(df)} rows for 2026.")
        return df

    except Exception as e:
        print(f"🛑 Error occurred: {e}")
    finally:
        driver.quit()

# Run it
df_2026 = get_2026_data_resilient()

Opening https://peerlist.io/layoffs-tracker...
✅ Success! Scraped 4 rows for 2026.


Merging 2 Datasets

In [15]:
df_hist_clean = df_hist.rename(columns={
    'total_laid_off': 'Laid_Off_Count',
    'country': 'Country',
    'date': 'Date',
    'company': 'Company',
    'industry' : 'Industry'
})

df_2026_clean = df_2026.rename(columns={
    'Layoff_Count': 'Laid_Off_Count',
    'Location': 'Country'
})

In [16]:
df_2026_clean['Laid_Off_Count'] = df_2026_clean['Laid_Off_Count'].astype(str).str.split(' ').str[0]

# 2. Convert counts to numbers (float handles 'NaN' better than int)
df_hist_clean['Laid_Off_Count'] = pd.to_numeric(df_hist_clean['Laid_Off_Count'], errors='coerce')
df_2026_clean['Laid_Off_Count'] = pd.to_numeric(df_2026_clean['Laid_Off_Count'], errors='coerce')

# 3. Standardize Dates
df_hist_clean['Date'] = pd.to_datetime(df_hist_clean['Date'])
df_2026_clean['Date'] = pd.to_datetime(df_2026_clean['Date'])

In [17]:
df_master = pd.concat([df_hist_clean, df_2026_clean], ignore_index=True)

# Drop any potential duplicates (if a 2025 layoff was in both sets)
df_master = df_master.drop_duplicates(subset=['Company', 'Date', 'Laid_Off_Count'])

# Sort by date to see the timeline clearly
df_master = df_master.sort_values(by='Date', ascending=False)

# Save your final "Single Source of Truth"
df_master.to_csv("Layoffs_2022-2026.csv", index=False)

print(f"Merge Complete! Total records in master dataset: {len(df_master)}")

Merge Complete! Total records in master dataset: 4251
